<a href="https://colab.research.google.com/github/AbdullahHasan0/AI-Powered-PDF-Question-Answering-RAG-Pipeline-/blob/main/Resume_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resume Enhancer

Important: OpenAI API Key Required

This project uses OpenAI models for analyzing resumes and generating personalized cover letters.  
When running on **Google Colab**, you need to provide your OpenAI API key.

# Install necessary libraries if running for the first time


In [ ]:
%pip install openai ipython pydantic PyPDF2


In [ ]:
# Let's install and import Pydantic
# In Pydantic, BaseModel is the core class that you use to create data models.
# BaseModel is like a blueprint for structured data. It defines the fields, their types, and automatically gives you data validation and type conversion capabilities
from pydantic import BaseModel

# Importing other necessary libraries
import os
from openai import OpenAI  # Make sure you have the latest openai package (pip install --upgrade openai)
import json

from typing import List, Dict, Union, Optional, Any
from IPython.display import display, Markdown

from google.colab import userdata
print("Libraries imported successfully!")



In [ ]:


# Fetch API keys from environment variables
openai_api_key = userdata.get("OPENAI_API_KEY")


# Configure the APIs
openai_client = OpenAI(api_key = openai_api_key)


# DEFINING THE LLM INPUTS INCLUDING RESUME AND TARGET JOB DESCRIPTION

- We need to get the user's resume and the target job description.


In [ ]:
# Helper function to display markdown nicely
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))

In [ ]:
from google.colab import files
from PyPDF2 import PdfReader
import os

def pick_and_extract_pdf(save_to="store"):
    """
    Opens a file picker to upload a single PDF, extracts its text,
    and saves it into a .txt file.

    Args:
        save_to (str): Folder to save extracted .txt file (default: 'store').

    Returns:
        str: Extracted text
    """
    # File picker
    uploaded = files.upload()
    pdf_path = list(uploaded.keys())[0]

    # Make sure output folder exists
    if not os.path.exists(save_to):
        os.makedirs(save_to)

    # Read PDF
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""

    # Save text into a .txt file
    filename = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(save_to, filename)

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"✅ Extracted and saved text to: {txt_path}")

    return text


In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_file_path: str) -> str:
    text = ""
    with open(pdf_file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text.strip()


In [ ]:
resume_text = pick_and_extract_pdf()

In [ ]:
# Let's define a sample job description text
job_description_text = """
We are looking for a motivated and enthusiastic AI/ML Intern to join our team and work on cutting-edge projects. This is an excellent opportunity to apply your skills in machine learning, data science, and programming to solve real-world challenges.

Key Responsibilities:

Work with the team to design and improve machine learning models for specific projects.
Clean and prepare data for analysis and model training.
Explore and test new machine learning techniques to make models more accurate and efficient.
Help deploy and test models in environments similar to production.
Collaborate with software engineers to integrate models into existing systems.
Stay updated on new AI and machine learning tools and trends
 Keep detailed records of processes, findings, models, and performance for future reference.


Requirements:

A degree in Computer Science, Data Science, or a related field is required.
Good understanding of machine learning algorithms like regression, classification, and clustering.
Proficient in Python and libraries
Familiar with cloud platforms like AWS, Google Cloud, or Azure.
Knowledge of NLP or computer vision is a plus.
Comfortable with version control systems, particularly Git.


What We Offer:

A supportive environment with mentorship opportunities to grow your skills.
Opportunities to work on impactful projects and contribute to innovative products.
Access to learning resources and professional development.
"""

In [ ]:
# Let's display the original resume
print_markdown("**--- Original Resume ---**")
print_markdown(resume_text)

In [ ]:
# Let's display the target job desciption
print_markdown("\n**--- Target Job Description ---**")
print_markdown(job_description_text)

# ENHANCING THE RESUME WITH OPENAI API

Now that we have our resume and job description, let's use the OpenAI API to improve the resume to better match the job requirements. We'll make a call to the text generation API and ask it to enhance our resume.


In [ ]:
def openai_generate(prompt: str,
                    model: str = "gpt-4o-mini",
                    temperature: float = 0.7,
                    max_tokens: int = 1500,
                    response_format: Optional[dict] = None) -> str | dict:
    """
    Generate text using OpenAI API

    This function sends a prompt to OpenAI's API and returns the generated response.
    It supports both standard text generation and structured parsing with response_format.

    Args:
        prompt (str): The prompt to send to the model, i.e.: your instructions for the AI
        model (str): The OpenAI model to use (default: "gpt-4o-mini")
        temperature (float): Controls randomness, where lower values make output more deterministic
        max_tokens (int): Maximum number of tokens to generate, which limits the response length
        response_format (dict): Optional format specification
        In simple terms, response_format is optional. If the user gives me a dictionary, cool!
        If they don't give me anything, just assume it's None and keep going."

    Returns:
        str or dict: The generated text or parsed structured data, depending on response_format
    """


    try:
        # Standard text generation without a specific response format
        if not response_format:
            response = openai_client.chat.completions.create(
                model = model,
                messages = [
                    {"role": "system",
                     "content": "You are a helpful assistant specializing in resume writing and career advice.",
                    },
                    {"role": "user", "content": prompt}],
                temperature = temperature,
                max_tokens = max_tokens)

            # Extract just the text content from the response
            return response.choices[0].message.content

        # Structured response generation (e.g., JSON format)
        else:
            completion = openai_client.beta.chat.completions.parse(
                model = model,  # Make sure to use a model that supports parse
                messages = [
                    # Same system and user messages as above
                    {
                        "role": "system",
                        "content": "You are a helpful assistant specializing in resume writing and career advice.",
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature = temperature,
                response_format = response_format)

            # Return the parsed structured output
            return completion.choices[0].message.parsed

    except Exception as e:
        # Error handling to prevent crashes
        return f"Error generating text: {e}"


In [ ]:
prompt = f"""
Context:
You are a professional resume writer helping a candidate tailor their resume for a specific job opportunity. The resume and job description are provided below.

Instruction:
Enhance the resume to make it more impactful. Focus on:
- Highlighting relevant skills and achievements.
- Using strong action verbs and quantifiable results where possible.
- Rewriting vague or generic bullet points to be specific and results-driven.
- Emphasizing experience and skills most relevant to the job description.
- Reorganizing sections if necessary to better match the job requirements.

Resume:
{resume_text}

Output:
Provide a revised and improved version of the resume that is well-formatted. Only return the updated resume.
"""


In [ ]:
# Get response from OpenAI API
openai_output = openai_generate(prompt, temperature = 0.7)

# Display the results
print_markdown("#### OpenAI Response:")
print_markdown(openai_output)

# PERFORMING A GAP ANALYSIS BETWEEN RESUME & JOB DESCRIPTION

Now, let's use an LLM to analyze the resume and job description. We want the AI to identify:
1.  Key skills/requirements mentioned in the Job Description.
2.  Relevant skills/experience present in the Resume.
3.  Crucially, the mismatches or gaps – what the job asks for that the resume doesn't highlight well.

We'll use OpenAI for this analysis step. We need to craft a clear prompt.

In [ ]:
# Prompt to analyze the resume against the job description

def analyze_resume_against_job_description(job_description_text: str, resume_text: str, model: str = "openai") -> str:
    """
    Analyze the resume against the job description and return a structured comparison.

    Args:
        job_description_text (str): The job description text.
        resume_text (str): The candidate's resume text.
        model (str): The model to use for analysis ("openai" or "gemini").

    Returns:
        str: A clear, structured comparison of the resume and job description.
    """
    # This prompt instructs the AI to act as a career advisor and analyze how well the resume matches the job description
    # It asks for a structured analysis with 4 specific sections: requirements, matches, gaps, and strengths
    prompt = f"""
    Context:
    You are a career advisor and resume expert. Your task is to analyze a candidate's resume against a specific job description to assess alignment and identify areas for improvement.

    Instruction:
    Review the provided Job Description and Resume. Identify key skills, experiences, and qualifications in the Job Description and compare them to what's present in the Resume. Provide a structured analysis with the following sections:
    1. **Key Requirements from Job Description:** List the main skills, experiences, and qualifications sought by the employer.
    2. **Relevant Experience in Resume:** List the skills and experiences from the resume that match or align closely with the job requirements.
    3. **Gaps/Mismatches:** Identify important skills or qualifications from the Job Description that are missing, unclear, or underrepresented in the Resume.
    4. **Potential Strengths:** Highlight any valuable skills, experiences, or accomplishments in the resume that are not explicitly requested in the job description but could strengthen the application.

    Job Description:

    {job_description_text}

    Resume:

    {resume_text}

    Output:
    Return a clear, structured comparison with the four sections outlined above.
    """

    # This conditional block selects which AI model to use based on the 'model' parameter
    if model == "openai":
        # Uses OpenAI's model to generate the gap analysis with moderate creativity (temperature=0.7)
        gap_analysis = openai_generate(prompt, temperature=0.7)
    elif model == "gemini":
        # Uses Google's Gemini model with less creativity (temperature=0.5) for more focused results
        gap_analysis = gemini_generate(prompt, temperature=0.5)
    else:
        # Raises an error if an invalid model name is provided
        raise ValueError(f"Invalid model: {model}")

    # Returns the generated gap analysis text
    return gap_analysis



In [ ]:
# Call the function to analyze the resume against the job description using OpenAI
gap_analysis_openai = analyze_resume_against_job_description(job_description_text,
                                                             resume_text,
                                                             model = "openai")

# Displays the analysis results in Markdown format
print_markdown("#### OpenAI Response:")
print_markdown(gap_analysis_openai)

# DRAFTING A NEW TAILORED RESUME BY AI WITH CHANGE TRACKING (WITH PYDANTIC)

Now, we'll use the insights gained (analysis and suggestions) to generate a completely rewritten, tailored resume using OpenAI. A key enhancement here is to ask the AI not just to list the changes, but to try and identify which sections or areas of the resume were modified. This helps the user quickly see the impact of the tailoring.

We'll ask the AI for two outputs:
1.  The full text of the tailored resume.
2.  A structured list (using Markdown) describing the key changes and where they were made (e.g., Summary, Experience section, Skills).

In [ ]:
# Define Pydantic models for structured output
# The ResumeOutput class is a Pydantic model that defines the structure of the output
# for the resume generation function. It includes two fields:
# (1) updated_resume: A string that contains the final rewritten resume.
# (2) diff_markdown: A string containing the resume's HTML-coloured version highlighting additions and deletions.

class ResumeOutput(BaseModel):
    updated_resume: str
    diff_markdown: str


def generate_resume(
    job_description_text: str, resume_text: str, gap_analysis_openai: str, model: str = "openai") -> dict:
    """
    Generate a tailored resume using OpenAI or Gemini.

    Args:
        job_description_text (str): The job description text.
        resume_text (str): The candidate's resume text.
        gap_analysis_openai (str): The gap analysis result from OpenAI.
        model (str): The model to use for resume generation.

    Returns:
        dict: A dictionary containing the updated resume and diff markdown.
    """
    # Construct the prompt for the AI model to generate the tailored resume.
    # The prompt includes context, instructions, and input data (original resume,
    # target job description, and gap analysis).
    prompt = (
        """
    ### Context:
    You are an expert resume writer and editor. Your goal is to rewrite the original resume to match the target job description, using the provided tailoring suggestions and analysis.

    ---

    ### Instruction:
    1. Rewrite the entire resume to best match the **Target Job Description** and **Gap Analysis to the Job Description**.
    2. Improve clarity, add job-relevant keywords, and quantify achievements.
    3. Specifically address the gaps identified in the analysis by:
       - Adding missing skills and technologies mentioned in the job description
       - Reframing experience to highlight relevant accomplishments
       - Strengthening sections that were identified as weak in the analysis
    4. Prioritize addressing the most critical gaps first
    5. Incorporate industry-specific terminology from the job description
    6. Ensure all quantifiable achievements are properly highlighted with metrics
    7. Return two versions of the resume:
        - `updated_resume`: The final rewritten resume (as plain text)
        - `diff_html`: A version of the resume with inline highlights using color:
            - Additions or rewritten content should be **green**:
            `<span style="color:green">your added or changed text</span>`
            - Removed content should be **red and struck through**:
            `<span style="color:red;text-decoration:line-through">removed text</span>`
            - Leave unchanged lines unmarked.
        - Keep all section headers and formatting consistent with the original resume.

    ---

    ### Output Format:

    ```json
    {
    "updated_resume": "<full rewritten resume as plain text>",
    "diff_markdown": "<HTML-colored version of the resume highlighting additions and deletions>"
    }
    ```
    ---
    ### Input:

    **Original Resume:**

    """
        + resume_text
        + """


    **Target Job Description:**

    """
        + job_description_text
        + """


    **Analysis of Resume vs. Job Description:**

    """
        + gap_analysis_openai
    )

    # Depending on the selected model, call the appropriate function to generate the resume.
    # If the OpenAI model is selected, it uses a temperature of 0.7 for creativity.
    if model == "openai":
        updated_resume_json = openai_generate(prompt, temperature = 0.7, response_format = ResumeOutput)
    # If the Gemini model is selected, it uses a lower temperature of 0.5 for more focused results.
    elif model == "gemini":
        updated_resume_json = gemini_generate(prompt, temperature = 0.5)
    else:
        # Raise an error if an invalid model name is provided.
        raise ValueError(f"Invalid model: {model}")

    # Return the generated resume output as a dictionary.
    return updated_resume_json


In [ ]:
# Call the generate_resume function with the provided job description, resume text, and gap analysis.
updated_resume_json = generate_resume(job_description_text, resume_text, gap_analysis_openai, model="openai")
# Display the updated resume in Markdown format.
print_markdown(updated_resume_json.updated_resume)

In [ ]:
print_markdown(updated_resume_json.diff_markdown)

# GENERATING A CUSTOM COVER LETTER

With the newly tailored resume, let's generate a corresponding cover letter.
We'll use OpenAI, feeding it the tailored resume (from the previous task) and the original job description. This ensures the cover letter highlights the most relevant points from the improved resume.

In [ ]:
# Define Pydantic models for structured output
# The CoverLetterOutput class is a Pydantic model that defines the structure of the output for the cover letter generation.
# It ensures that the output will contain a single field, 'cover_letter', which is a string.

class CoverLetterOutput(BaseModel):
    cover_letter: str

# The generate_cover_letter function creates a cover letter based on the provided job description and updated resume.
# It takes three parameters:
# (1) job_description_text: A string containing the job description for the position.
# (2) updated_resume: A string containing the candidate's updated resume.
# (3) model: A string indicating which model to use for generating the cover letter (default is "openai").
# The function returns a dictionary containing the generated cover letter.

def generate_cover_letter(job_description_text: str, updated_resume: str, model: str = "openai") -> dict:
    """
    Generate a cover letter using OpenAI or Gemini.

    Args:
        job_description_text (str): The job description text.
        updated_resume (str): The candidate's updated resume text.
        model (str): The model to use for cover letter generation.

    Returns:
        dict: A dictionary containing the cover letter.
    """

    # Construct the prompt for the AI model, including context and instructions for writing the cover letter.
    prompt = (
        """
    ### Context:
    You are a professional career coach and expert cover letter writer.

    ---

    ### Instruction:
    Write a compelling, personalized cover letter based on the **Updated Resume** and the **Target Job Description**. The letter should:
    1. Be addressed generically (e.g., "Dear Hiring Manager")
    2. Be no longer than 4 paragraphs
    3. Highlight key achievements and experiences from the updated resume
    4. Align with the responsibilities and qualifications in the job description
    5. Reflect the applicant's enthusiasm and fit for the role
    6. End with a confident and polite closing statement

    ---

    ### Output Format (JSON):
    ```json
    {
    "cover_letter": "<final cover letter text>"
    }
    ```
    ---

    ### Input:

    **Updated Resume:**

    """
        + updated_resume
        + """
    **Target Job Description:**

    """
        + job_description_text
    )

    # Depending on the selected model, call the appropriate function to generate the cover letter.
    if model == "openai":
        # Get response from OpenAI API
        updated_cover_letter = openai_generate(prompt, temperature=0.7, response_format=CoverLetterOutput)
    elif model == "gemini":
        # Get response from Gemini API
        updated_cover_letter = gemini_generate(prompt, temperature=0.5)
    else:
        # Raise an error if an invalid model name is provided.
        raise ValueError(f"Invalid model: {model}")

    # Return the generated cover letter as a dictionary.
    return updated_cover_letter


In [ ]:
# Call the generate_cover_letter function with the provided job description and updated resume.
updated_cover_letter = generate_cover_letter(job_description_text, updated_resume_json.updated_resume, model="openai")

# Display the generated cover letter in Markdown format.
print_markdown(updated_cover_letter.cover_letter)

# UNIFIENING RESUME AND COVER LETTER GENERATION FUNCTION

Now that we have all the building blocks, let's create a single function, `run_resume_rocket`, that takes the original resume and job description text and performs the entire workflow: gap analysis, resume tailoring with diff tracking, and cover letter generation. This makes the tool much easier to reuse.

In [ ]:


def run_resume_rocket(resume, job_description_text: str) -> tuple[str, str]:
    """
    Run the resume rocket workflow.

    Args:
        resume_text (str): The candidate's resume text.
        job_description_text (str): The job description text.

    Returns:
        tuple: A tuple containing the updated resume and cover letter.
    """

    # --- Validation: Both inputs required ---
    if resume is None:
        return "⚠️ Please upload a resume file.", ""
    if not job_description_text.strip():
        return "", "⚠️ Please paste the job description."



    # Analyze the candidate's resume against the job description using OpenAI's model.
    # This function will return a structured analysis highlighting gaps and strengths.



     # Detect file type
    file_path = resume.name
    if file_path.endswith(".pdf"):
        resume_text = extract_text_from_pdf(file_path)
    else:
        # fallback for txt or unknown
        with open(file_path, "r", encoding="utf-8") as f:
            resume_text = f.read()


    gap_analysis_openai = analyze_resume_against_job_description(job_description_text,
                                                                 resume_text,
                                                                 model="openai")

    # Display the gap analysis results in Markdown format for better readability.
    print_markdown(gap_analysis_openai)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Generate an updated resume based on the job description, original resume, and gap analysis.
    # This function will return a JSON-like object containing the updated resume and a diff markdown.
    updated_resume_json = generate_resume(job_description_text,
                                          resume_text,
                                          gap_analysis_openai,
                                          model = "openai")

    # Display the diff markdown which shows the changes made to the resume.
    print_markdown(updated_resume_json.diff_markdown)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Display the updated resume in Markdown format.
    print_markdown(updated_resume_json.updated_resume)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Generate a cover letter based on the job description and the updated resume.
    # This function will return the generated cover letter.
    updated_cover_letter = generate_cover_letter(
        job_description_text, updated_resume_json.updated_resume, model="openai"
    )

    # Display the generated cover letter in Markdown format.
    print_markdown(updated_cover_letter.cover_letter)

    # Print separators for clarity in the output.
    print("\n--------------------------------")
    print("--------------------------------\n")

    # Return the updated resume and the generated cover letter as a tuple.
    return updated_resume_json.updated_resume, updated_cover_letter.cover_letter



In [ ]:
# # Call the run_resume_rocket function with the provided resume and job description texts.
# resume, cover_letter = run_resume_rocket(resume_text, job_description_text)

# Making Gradio App

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Ocean()) as demo:
    gr.Markdown("# Resume Maker")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Upload Resume & Job Description\nGet an **Updated Resume** and a **Personalized Cover Letter**")

            resume_upload = gr.File(label="Upload Resume", file_types=[".pdf", ".docx", ".txt"])
            job_desc = gr.TextArea(label="Paste Job Description Here", lines=5, placeholder="Paste the job description...")

            generate_btn = gr.Button("Generate")

        with gr.Column(scale=1):
            updated_resume = gr.Textbox(label="Updated Resume", lines=15, interactive=False)
            cover_letter = gr.Textbox(label="Cover Letter", lines=15, interactive=False)

    generate_btn.click(
        fn=run_resume_rocket,
        inputs=[resume_upload, job_desc],
        outputs=[updated_resume, cover_letter]
    )

demo.launch()